In [1]:
"""
Tutoriel SQLAlchemy pour se connecter à une base de données MariaDB
et effectuer des opérations CRUD de base.
"""

import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from sqlalchemy import text


In [2]:
# Configuration de la connexion
# Remplacez 'prenom' par votre prénom et 'date_de_naissance' par votre date de naissance
# Format recommandé pour la date de naissance: AAAAMMJJ ou le format spécifié par l'administrateur
username = 'kevin'  # Remplacez par votre prénom
password = '18/02/1989'  # Remplacez par votre date de naissance
host = 'beta-seed.unistra.fr'
port = '3306'
database = 'notation'

In [21]:
# Créer l'URL de connexion
connection_url = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
# Créer un moteur SQLAlchemy
engine = sa.create_engine(connection_url)  # echo=True affiche les requêtes SQL exécutées

In [22]:
connection = engine.connect()
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['cours', 'etudiants', 'inscriptions', 'log_actions', 'notes', 'projet']

In [23]:
table_name = 'etudiants'
columns = inspector.get_columns(table_name)
print(f"  Colonnes dans {table_name}:")
for column in columns:
    print(f"    - {column['name']} ({column['type']})")

  Colonnes dans etudiants:
    - id (INTEGER)
    - nom (VARCHAR(50))
    - prenom (VARCHAR(50))
    - email (VARCHAR(50))


In [26]:
# SELECT
query = """
        SELECT *
        FROM etudiants; \
        """

# execute() - Exécute une requête SQL et retourne un curseur pour récupérer les résultats
# fetchall() - Récupère toutes les lignes restantes d'un résultat de requête sous forme de liste de tuples
# fetchone() - Récupère la prochaine ligne du résultat sous forme de tuple, retourne None s'il n'y en a plus

with engine.connect() as connection:
    result_cursor = connection.execute(text(query))
    # Le résultat est un objet CursorResult qui contient les lignes de résultats
    # Chaque ligne est un objet Row qui peut être traité comme un tuple ou un dictionnaire

results = result_cursor.fetchall()  # Récupère toutes les lignes
print(f"{type(results)=}")
print(f"{results=}")



type(results)=<class 'list'>
results=[(4, 'Michoud', 'Kevin', 'kmichoud@unistra.fr'), (5, 'Michoud', 'Sacha', 'too_young_for_email@email.com'), (9, 'MICHOUD', 'Helen', 'her_email@email.com'), (10, 'MICHOUD', 'Helen', 'her_email@email.com')]


In [25]:
# INSERT
query = """
        INSERT INTO etudiants (nom, prenom, email)
        VALUES ('MICHOUD', 'Helen', 'her_email@email.com') \
        """
with engine.connect() as connection:
    result = connection.execute(text(query))
    # IF connect, we need to commit the changes
    connection.commit()  # Add commit to persist the changes



In [27]:
# DELETE FROM
query = """
DELETE FROM etudiants WHERE prenom='Helen' and nom='MICHOUD';
"""
with engine.begin() as connection:
    # If begin, we don't need to commit the changes'
    result = connection.execute(text(query))


In [28]:
# UPDATE
query = """
UPDATE etudiants
SET prenom='Sacha' WHERE id= '5';
"""
with engine.begin() as connection:
    result = connection.execute(text(query))

In [ ]:
# Définition des anciens et nouveaux domaines emails
old_email = "email.com"  # Le domaine email qu'on veut remplacer
new_email = "google.com"  # Le nouveau domaine email

# Création d'une requête SQL paramétrée pour mettre à jour les emails
query = text("""
             UPDATE etudiants # Met à jour la table étudiants
             SET email = REPLACE(email, : old, : new) # Remplace l ancien domaine par le nouveau
WHERE email LIKE :email_pattern                     # Filtre les emails qui contiennent l ancien domaine
             """)  # Les :old, :new sont des paramètres qui seront remplis plus tard

# Exécution de la requête avec une connexion automatiquement fermée
with engine.begin() as connection:  # Ouvre une connexion à la base de données
    result = connection.execute(query,  # Exécute la requête SQL
                                {"old": old_email,  # Paramètre pour l'ancien domaine
                                 "new": new_email,  # Paramètre pour le nouveau domaine
                                 "email_pattern": f"%{old_email}"})  # Crée le motif de recherche
    print(f"Number of rows updated: {result.rowcount}")  # Affiche le nombre de lignes modifiées


In [30]:
import pandas as pd

df = pd.read_sql_table("etudiants", engine)
df

,id,nom,prenom,email
0,4,Michoud,Kevin,kmichoud@unistra.fr
1,5,Michoud,Sacha,too_young_for_email@email.com
